In [217]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv("csv/sql/processed.csv", dtype= {'teamid': 'str'})
blue_side = df[df["side"] == 100]
blue_side = blue_side.rename(columns={
    "teamid": "blue_teamid",
})
blue_side.drop(columns=["side"], inplace=True)

red_side = df[df["side"] == 200]
red_side = red_side.rename(columns={
    "teamid": "red_teamid",
})
red_side.drop(columns=["side", "esportsgameid"], inplace=True)

df = blue_side.merge(red_side, on="platformgameid", suffixes=("_blue", "_red"))
df['winningteam'] = np.where(df['win_blue'] == 1, 100, 200)
df.drop(columns=["win_blue", "win_red"], inplace=True)
display(df.head())

df.fillna(0, inplace=True)
features = [
    "first_blood_avg",
    "first_inhibitor_avg",
    "first_tower_avg",
    "kills_avg",
    "win_avg",
    "deaths_avg",
    "level_avg",
    "cs_avg",
]

def gaus_test(features, df):
    X = df[[i+"_blue" for i in features] + [i+"_red" for i in features ]]
    y = df["winningteam"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

    gnb = GaussianNB()
    model = gnb.fit(X_train,y_train)
    predictive_labels = gnb.predict(X_test)

    return round(accuracy_score(y_test, predictive_labels), 3)

def gaus_test_iter(features, df, n):
    acc = 0
    for i in range(n):
        acc += gaus_test(features, df)
    return acc/n

gaus_test_iter(features, df, 100)

,blue_teamid,esportsgameid,platformgameid,first_blood_time_blue,first_blood_blue,first_tower_time_blue,first_inhibitor_time_blue,first_tower_blue,first_inhibitor_blue,kills_blue,...,deaths_avg_red,level_avg_red,cs_avg_red,first_blood_avg_red,first_tower_avg_red,first_inhibitor_avg_red,first_blood_time_avg_red,first_tower_time_avg_red,first_inhibitor_time_avg_red,winningteam
0,100205573495116443,102181142519049888,ESPORTSTMNT01:1130786,22.697100,1,1048277,1972924,0,0,5,...,5.000000,17.0,60.000000,0.0,1.000000,1.000000,22.697100,1.048277e+06,1.972924e+06,200
1,100205573495116443,102181142519246497,ESPORTSTMNT01:1130806,9.405167,1,781709,1977250,1,0,4,...,4.500000,16.5,56.500000,0.0,0.500000,1.000000,16.051133,9.149930e+05,1.975087e+06,200
2,100205573495116443,102181142519377570,ESPORTSTMNT01:1140917,9.104283,1,1166569,1915734,1,1,16,...,8.333333,16.0,57.333333,0.0,0.333333,0.666667,13.735517,9.988517e+05,1.955303e+06,100
3,100205573495116443,102181142541397833,ESPORTSTMNT01:1143282,3.755650,1,769258,1844212,0,1,13,...,14.600000,15.2,60.200000,0.0,0.800000,0.400000,10.965870,9.310408e+05,1.970765e+06,100
4,100205573495116443,102181142541463370,ESPORTSTMNT01:1143414,6.889967,1,893223,1445342,1,0,10,...,13.200000,14.8,59.000000,0.0,0.600000,0.400000,11.691720,9.131044e+05,1.914671e+06,200


0.79285

In [214]:
import itertools

for comb in itertools.permutations(features, 5):
    print(comb)
    acc = gaus_test_iter(comb, df, 100)
    if acc > 0.8:
        break
    print(acc)

('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'kills_avg', 'win_avg')
0.7920599999999999
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'kills_avg', 'deaths_avg')
0.7755000000000005
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'kills_avg', 'level_avg')
0.7694500000000002
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'kills_avg', 'cs_avg')
0.7685000000000004
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'win_avg', 'kills_avg')
0.7915000000000001
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'win_avg', 'deaths_avg')
0.79074
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'win_avg', 'level_avg')
0.7910300000000001
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'win_avg', 'cs_avg')
0.7902300000000002
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg', 'deaths_avg', 'kills_avg')
0.7762400000000005
('first_blood_avg', 'first_inhibitor_avg', 'first_tower_avg',

In [216]:
X = df[[i+"_blue" for i in features] + [i+"_red" for i in features ]]
y = df["winningteam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

gnb = GaussianNB()
model = gnb.fit(X_train,y_train)
gnb.predict_proba(X_test)

array([[9.93440865e-01, 6.55913474e-03],
       [6.59680770e-02, 9.34031923e-01],
       [9.86327201e-01, 1.36727990e-02],
       ...,
       [6.66320668e-01, 3.33679332e-01],
       [7.95098342e-05, 9.99920490e-01],
       [1.98258040e-01, 8.01741960e-01]])